```{contents}
```
## Short-Term Memory

In LangGraph, **Short-Term Memory (STM)** is the **in-graph, ephemeral state** that exists **during the execution of a workflow** and is shared across nodes. It is the core mechanism by which LangGraph enables **context propagation, coordination, iterative reasoning, and control flow**.

---

### **1. Conceptual Definition**

> **Short-Term Memory = Graph State**

It is the **authoritative working memory** of the graph.

| Property    | Description                      |
| ----------- | -------------------------------- |
| Scope       | One graph execution (one thread) |
| Lifetime    | From invocation → termination    |
| Mutability  | Updated incrementally by nodes   |
| Visibility  | Shared across all nodes          |
| Persistence | Optional via checkpointing       |

---

### **2. Why STM Is Essential**

Without STM, nodes are isolated function calls.
With STM, the graph becomes a **coherent cognitive system**.

| Capability           | Enabled by STM                  |
| -------------------- | ------------------------------- |
| Multi-step reasoning | Accumulate intermediate results |
| Agent coordination   | Shared context                  |
| Loop control         | Track progress and termination  |
| Tool orchestration   | Pass results between tools      |
| Human-in-the-loop    | Inject corrections              |
| Autonomy             | Maintain evolving plan          |

---

### **3. How STM Is Represented**

STM is defined by a **typed state schema**.

```python
from typing import TypedDict, List

class State(TypedDict):
    messages: List[str]
    plan: str
    result: str
    step: int
```

This schema is the **contract** between all nodes.

---

### **4. STM Lifecycle**

```
Initialize → Update → Merge → Read → Update → … → Finalize
```

Each node:

1. **Reads** current state
2. Performs computation
3. **Returns partial update**
4. LangGraph **merges updates** into global state

---

### **5. Partial State Updates & Reducers**

Nodes never replace the entire memory—only the fields they modify.

```python
def planner(state):
    return {"plan": "Search then summarize"}
```

Reducers control **how updates merge**:

```python
from langgraph.graph import add_messages

class State(TypedDict):
    messages: list

State = Annotated[State, {"messages": add_messages}]
```

This allows **append-only memory**, conflict resolution, and controlled mutation.

---

### **6. STM in Cyclic & Agentic Systems**

STM enables **loop intelligence**:

```python
def should_continue(state):
    return "finish" if state["step"] > 5 else "reason"
```

The state itself **controls execution**.

---

### **7. STM vs Long-Term Memory**

| Dimension   | Short-Term Memory      | Long-Term Memory        |
| ----------- | ---------------------- | ----------------------- |
| Lifetime    | Single execution       | Across executions       |
| Storage     | In-memory / checkpoint | Database / vector store |
| Purpose     | Reasoning & control    | Knowledge retention     |
| Speed       | Very fast              | Slower                  |
| Consistency | Strong                 | Eventual                |

---

### **8. Example: STM in a Reasoning Loop**

```python
from langgraph.graph import StateGraph, END

class State(TypedDict):
    thought: str
    step: int

def reason(state):
    return {"thought": f"Step {state['step']}", "step": state["step"] + 1}

def route(state):
    return END if state["step"] >= 3 else "reason"

g = StateGraph(State)
g.add_node("reason", reason)
g.set_entry_point("reason")
g.add_conditional_edges("reason", route, {"reason": "reason", END: END})

graph = g.compile()
print(graph.invoke({"thought": "", "step": 0}))
```

The evolving `State` **is the short-term memory**.

---

### **9. Production Considerations**

| Concern        | Solution                    |
| -------------- | --------------------------- |
| State growth   | Prune / summarize           |
| Large memory   | Offload to long-term memory |
| Crash recovery | Checkpoint store            |
| Concurrency    | Thread-scoped STM           |
| Debugging      | State inspection tools      |

---

### **10. Mental Model**

LangGraph STM functions exactly like:

> **Working memory in the human brain**

It holds:

* current thoughts
* plans
* observations
* intermediate conclusions

and evolves continuously until the task completes.

### Demonstration

In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from typing_extensions import Annotated
from langgraph.graph import add_messages


In [2]:
class State(TypedDict):
    thoughts: Annotated[List[str], add_messages]
    step: int

def reason(state: State):
    new_thought = f"Reasoning at step {state['step']}"
    return {
        "thoughts": [new_thought],
        "step": state["step"] + 1
    }

def decide(state: State):
    if state["step"] >= 4:
        return END
    return "reason"



In [3]:
builder = StateGraph(State)

builder.add_node("reason", reason)
builder.set_entry_point("reason")

builder.add_conditional_edges("reason", decide, {
    "reason": "reason",
    END: END
})

graph = builder.compile()


In [4]:
result = graph.invoke({"thoughts": [], "step": 0})
print(result)


{'thoughts': [HumanMessage(content='Reasoning at step 0', additional_kwargs={}, response_metadata={}, id='39b8182c-4155-4f8f-8dd6-f3ffb18c5657'), HumanMessage(content='Reasoning at step 1', additional_kwargs={}, response_metadata={}, id='9f0765a8-b162-4cbc-aa66-96eb642e85c7'), HumanMessage(content='Reasoning at step 2', additional_kwargs={}, response_metadata={}, id='f912833d-3c19-4e89-9542-1d50f9614282'), HumanMessage(content='Reasoning at step 3', additional_kwargs={}, response_metadata={}, id='1ee5ffac-1bcc-4c02-be35-f4512a87a111')], 'step': 4}
